SQL Queries & Java Programming by Mason Howes -- Please note, this program leverages starter code from [this github repository](https://github.com/eden-ski/vaccine-scheduler-java). This program also relies on conneting to an Azure database, which I will not be providing code for since that would be a bit of a hassle to set up, so I will save you the trouble and just showcase the SQL and Java code used.

This SQL code was originally ran with [SQLite 3.](https://www.sqlite.org/)

Password hashing (Java - Partially provided):

In [ ]:
package scheduler.util;

import javax.crypto.SecretKeyFactory;
import javax.crypto.spec.PBEKeySpec;
import java.security.NoSuchAlgorithmException;
import java.security.SecureRandom;
import java.security.spec.InvalidKeySpecException;
import java.security.spec.KeySpec;
import java.util.Arrays;

public class Util {

    // constants for handling password
    private static final int HASH_STRENGTH = 10;
    private static final int KEY_LENGTH = 16;

    public static byte[] generateSalt() {
        // Generate a random cryptographic salt
        SecureRandom random = new SecureRandom();
        byte[] salt = new byte[16];
        random.nextBytes(salt);
        return salt;
    }

    public static byte[] generateHash(String password, byte[] salt) {
        // Specify the hash parameters
        KeySpec spec = new PBEKeySpec(password.toCharArray(), salt, HASH_STRENGTH, KEY_LENGTH);

        // Generate the hash
        SecretKeyFactory factory = null;
        byte[] hash = null;
        try {
            factory = SecretKeyFactory.getInstance("PBKDF2WithHmacSHA1");
            hash = factory.generateSecret(spec).getEncoded();
        } catch (NoSuchAlgorithmException | InvalidKeySpecException ex) {
            throw new IllegalStateException();
        }
        return hash;
    }

    public static byte[] trim(byte[] bytes)
    {
        int i = bytes.length - 1;
        while (i >= 0 && bytes[i] == 0)
        {
            --i;
        }

        return Arrays.copyOf(bytes, i + 1);
    }
}


Data model creation (SQL):

In [ ]:
CREATE TABLE Caregivers (
    Username varchar(255),
    Salt BINARY(16),
    Hash BINARY(16),
    PRIMARY KEY (Username)
);

CREATE TABLE Availabilities (
    Time date,
    Username varchar(255) REFERENCES Caregivers,
    PRIMARY KEY (Time, Username)
);

CREATE TABLE Vaccines (
    Name varchar(255),
    Doses int,
    PRIMARY KEY (Name)
);

CREATE TABLE Patients (
    Username varchar(255),
    Salt BINARY(16),
    Hash BINARY(16),
    PRIMARY KEY (Username)
);

CREATE TABLE Reservations (
    Reserve_id int,
    Time date,
    C_Username varchar(255) REFERENCES Caregivers,
    P_Username varchar(255) REFERENCES Patients,
    V_Name varchar(255) REFERENCES Vaccines,
    PRIMARY KEY (Reserve_id)
);

Manages connection to Azure DB (Java/SQL - Partially provided):

In [ ]:
package scheduler.db;

import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.SQLException;

public class ConnectionManager {

    private final String driverName = "com.microsoft.sqlserver.jdbc.SQLServerDriver";
    private final String connectionUrl = "jdbc:sqlserver://" + System.getenv("Server") +
            ".database.windows.net:1433;database=" + System.getenv("DBName");
    private final String userName = System.getenv("UserID");
    private final String userPass = System.getenv("Password");

    private Connection con = null;

    public ConnectionManager() {
        try {
            Class.forName(driverName);
        } catch (ClassNotFoundException e) {
            System.out.println(e.toString());
        }
    }

    public Connection createConnection() {
        try {
            con = DriverManager.getConnection(connectionUrl, userName, userPass);
        } catch (SQLException e) {
            e.printStackTrace();
        }
        return con;
    }

    public void closeConnection() {
        try {
            this.con.close();
        } catch (SQLException e) {
            e.printStackTrace();
        }
    }
}

Fleshes out the caregiver data table (Java that works with the data, calls SQL queries inline to communicate with Azure database):

In [ ]:
package scheduler.model;

import scheduler.db.ConnectionManager;
import scheduler.util.Util;

import java.sql.*;
import java.util.Arrays;


public class Caregiver {
    private final String username;
    private final byte[] salt;
    private final byte[] hash;

    private Caregiver(CaregiverBuilder builder) {
        this.username = builder.username;
        this.salt = builder.salt;
        this.hash = builder.hash;
    }

    private Caregiver(CaregiverGetter getter) {
        this.username = getter.username;
        this.salt = getter.salt;
        this.hash = getter.hash;
    }

    // Getters
    public String getUsername() {
        return username;
    }

    public byte[] getSalt() {
        return salt;
    }

    public byte[] getHash() {
        return hash;
    }


    public void saveToDB() throws SQLException {
        ConnectionManager cm = new ConnectionManager();
        Connection con = cm.createConnection();

        String addCaregiver = "INSERT INTO Caregivers VALUES (? , ?, ?)";

        try {
            PreparedStatement statement = con.prepareStatement(addCaregiver);
            statement.setString(1, this.username);
            statement.setBytes(2, this.salt);
            statement.setBytes(3, this.hash);
            statement.executeUpdate();
        } catch (SQLException e) {
            throw new SQLException();
        } finally {
            cm.closeConnection();
        }
    }

    public void uploadAvailability(Date d) throws SQLException {
        ConnectionManager cm = new ConnectionManager();
        Connection con = cm.createConnection();

        String addAvailability = "INSERT INTO Availabilities VALUES (? , ?)";

        try {
            PreparedStatement statement = con.prepareStatement(addAvailability);
            statement.setDate(1, d);
            statement.setString(2, this.username);
            statement.executeUpdate();
        } catch (SQLException e) {
            throw new SQLException();
        } finally {
            cm.closeConnection();
        }
    }

    public static class CaregiverBuilder {

        private final String username;
        private final byte[] salt
        private final byte[] hash;

        public CaregiverBuilder(String username, byte[] salt, byte[] hash) {
            this.username = username;
            this.salt = salt;
            this.hash = hash;
        }

        public Caregiver build() {
            return new Caregiver(this);
        }
    }

    public static class CaregiverGetter {
        private final String username;
        private final String password;
        private byte[] salt;
        private byte[] hash;

        public CaregiverGetter(String username, String password) {
            this.username = username;
            this.password = password;
        }

        public Caregiver get() throws SQLException {

            ConnectionManager cm = new ConnectionManager();
            Connection con = cm.createConnection();

            String getCaregiver = "SELECT Salt, Hash FROM Caregivers WHERE Username = ?";

            try {
                PreparedStatement statement = con.prepareStatement(getCaregiver);
                statement.setString(1, this.username);
                ResultSet resultSet = statement.executeQuery();
                while (resultSet.next()) {

                    byte[] salt = resultSet.getBytes("Salt");

                    // we need to call Util.trim() to get rid of the paddings,
                    // try to remove the use of Util.trim() and you'll see :)
                    byte[] hash = Util.trim(resultSet.getBytes("Hash"));

                    // check if the password matches
                    byte[] calculatedHash = Util.generateHash(password, salt);

                    if (!Arrays.equals(hash, calculatedHash)) {
                        return null;
                    } else {
                        this.salt = salt;
                        this.hash = hash;
                        return new Caregiver(this);
                    }
                }

                return null;

            } catch (SQLException e) {

                throw new SQLException();

            } finally {

                cm.closeConnection();
            }
        }
    }
}

Patients and vaccines code have similar logic to the code provided above. The cells for these files will be at the bottom of the ipynb file. In practice this wouldn't be how they are organized, but since this code isn't going to be ran in this format I decided this to avoid a bunch of repeat code.

Commandline Scheduler (Java code that builds CMD user interface that takes data and runs it through the proper code, then builds SQL queries that can communicate with the Azure database. Either adds new data to the database, deletes existing data, reorganizes existing data, or returns what currently exists in it.):

In [ ]:
package scheduler;

import scheduler.db.ConnectionManager;
import scheduler.model.Caregiver;
import scheduler.model.Patient;
import scheduler.model.Vaccine;
import scheduler.util.Util;

import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;
import java.sql.Connection;
import java.sql.Date;
import java.sql.PreparedStatement;
import java.sql.ResultSet;
import java.sql.SQLException;

public class Scheduler {

    // Objects to keep track of the currently logged-in user
    // Note: At most one of currentCaregiver and currentPatient is not null
    // since only one user can be logged-in at a time
    private static Caregiver currentCaregiver = null;
    private static Patient currentPatient = null;

    public static void main(String[] args) {
        // Printing greetings text
        System.out.println();
        System.out.println("Welcome to the COVID-19 Vaccine Reservation Scheduling Application!");
        System.out.println("*** Please enter one of the following commands ***");
        System.out.println("> create_patient <username> <password>");
        System.out.println("> create_caregiver <username> <password>");
        System.out.println("> login_patient <username> <password>");
        System.out.println("> login_caregiver <username> <password>");
        System.out.println("> search_caregiver_schedule <date>");
        System.out.println("> reserve <date> <vaccine>");
        System.out.println("> upload_availability <date>");
        System.out.println("> cancel <appointment_id>");
        System.out.println("> add_doses <vaccine> <number>");
        System.out.println("> show_appointments");
        System.out.println("> logout");
        System.out.println("> quit");
        System.out.println();

        // Read input from user
        BufferedReader r = new BufferedReader(new InputStreamReader(System.in));
        while (true) {
            System.out.print("> ");
            String response = "";
            try {
                response = r.readLine();
            } catch (IOException e) {
                System.out.println("Please try again!");
            }
            // Split the user input by spaces
            String[] tokens = response.split(" ");
            // Check if input exists
            if (tokens.length == 0) {
                System.out.println("Please try again!");
                continue;
            }
            // Determine which operation to perform
            String operation = tokens[0];
            switch (operation) {
                case "create_patient":
                    createPatient(tokens);
                    break;
                case "create_caregiver":
                    createCaregiver(tokens);
                    break;
                case "login_patient":
                    loginPatient(tokens);
                    break;
                case "login_caregiver":
                    loginCaregiver(tokens);
                    break;
                case "search_caregiver_schedule":
                    searchCaregiverSchedule(tokens);
                    break;
                case "reserve":
                    reserve(tokens);
                    break;
                case "upload_availability":
                    uploadAvailability(tokens);
                    break;
                case "cancel":
                    cancel(tokens);
                    break;
                case "add_doses":
                    addDoses(tokens);
                    break;
                case "show_appointments":
                    showAppointments(tokens);
                    break;
                case "logout":
                    logout(tokens);
                    break;
                case "quit":
                    System.out.println("Bye!");
                    return;
                default:
                    System.out.println("Invalid operation name!");
            }
        }
    }

    private static void createPatient(String[] tokens) {

        if (tokens.length != 3) {
            System.out.println("Failed to create user.");
            return;
        }

        String username = tokens[1];
        String password = tokens[2];


        boolean lengthCheck = password.length() > 7;
        boolean upperCheck = false;
        boolean lowerCheck = false;
        boolean letterCheck = false;
        boolean numberCheck = false;
        boolean specialCheck = false;
        String specialCharacters = "!@#?";

        for (char curr : password.toCharArray()) {
            if (Character.isDigit(curr)) {
                numberCheck = true;
            } else if (Character.isUpperCase(curr)) {
                upperCheck = true;
                letterCheck = true;
            } else if (Character.isLowerCase(curr)) {
                lowerCheck = true;
                letterCheck = true;
            } else if (specialCharacters.indexOf(curr) >= 0) {
                specialCheck = true;
            }
        }

        if (!lengthCheck || !upperCheck || !lowerCheck || !letterCheck || !numberCheck || !specialCheck) {
            System.out.println("Invalid password, requires: ");
            if (!lengthCheck) System.out.println("- Length to exceed 8 characters");
            if (!upperCheck) System.out.println("- 1 or more uppercase letters");
            if (!lowerCheck) System.out.println("- 1 or more lowercase letters");
            if (!letterCheck) System.out.println("- 1 or more letters");
            if (!numberCheck) System.out.println("- 1 or more numbers");
            if (!specialCheck) System.out.println("- 1 or more !, @, #, or ? to be included");
            return;
        }

        if (usernameExistsPatient(username)) {
            System.out.println("Username taken, try again!");
            return;
        }

        byte[] salt = Util.generateSalt();
        byte[] hash = Util.generateHash(password, salt);

        try {
            Patient patient = new Patient.PatientBuilder(username, salt, hash).build();
            patient.saveToDB();
            System.out.println("Created user " + username);
        } catch (SQLException e) {
            System.out.println("Failed to create user.");
            e.printStackTrace();
        }
    }

    private static void createCaregiver(String[] tokens) {
        if (tokens.length != 3) {
            System.out.println("Failed to create user.");
            return;
        }

        String username = tokens[1];
        String password = tokens[2];

        // Checks pw strength
        boolean lengthCheck = password.length() > 7;
        boolean upperCheck = false;
        boolean lowerCheck = false;
        boolean letterCheck = false;
        boolean numberCheck = false;
        boolean specialCheck = false;
        String specialCharacters = "!@#?";

        for (char curr : password.toCharArray()) {
            if (Character.isDigit(curr)) {
                numberCheck = true;
            } else if (Character.isUpperCase(curr)) {
                upperCheck = true;
                letterCheck = true;
            } else if (Character.isLowerCase(curr)) {
                lowerCheck = true;
                letterCheck = true;
            } else if (specialCharacters.indexOf(curr) >= 0) {
                specialCheck = true;
            }
        }

        if (!lengthCheck || !upperCheck || !lowerCheck || !letterCheck || !numberCheck || !specialCheck) {
            System.out.println("Invalid password, requires: ");
            if (!lengthCheck) System.out.println("- Length to exceed 8 characters");
            if (!upperCheck) System.out.println("- 1 or more uppercase letters");
            if (!lowerCheck) System.out.println("- 1 or more lowercase letters");
            if (!letterCheck) System.out.println("- 1 or more letters");
            if (!numberCheck) System.out.println("- 1 or more numbers");
            if (!specialCheck) System.out.println("- 1 or more !, @, #, or ? to be included");
            return;
        }

        if (usernameExistsCaregiver(username)) {
            System.out.println("Username taken, try again!");
            return;
        }

        byte[] salt = Util.generateSalt();
        byte[] hash = Util.generateHash(password, salt);

        try {
            Caregiver caregiver = new Caregiver.CaregiverBuilder(username, salt, hash).build();
            caregiver.saveToDB();
            System.out.println("Created user " + username);
        } catch (SQLException e) {
            System.out.println("Failed to create user.");
            e.printStackTrace();
        }
    }

    private static boolean usernameExistsPatient(String username) {
        ConnectionManager cm = new ConnectionManager();
        Connection con = cm.createConnection();

        String selectUsername = "SELECT * FROM Patients WHERE Username = ?";
        try {
            PreparedStatement statement = con.prepareStatement(selectUsername);
            statement.setString(1, username);
            ResultSet resultSet = statement.executeQuery();
            return resultSet.isBeforeFirst();
        } catch (SQLException e) {
            System.out.println("Error occurred when checking username");
            e.printStackTrace();
        } finally {
            cm.closeConnection();
        }
        return true;
    }

    private static boolean usernameExistsCaregiver(String username) {
        ConnectionManager cm = new ConnectionManager();
        Connection con = cm.createConnection();

        String selectUsername = "SELECT * FROM Caregivers WHERE Username = ?";
        try {
            PreparedStatement statement = con.prepareStatement(selectUsername);
            statement.setString(1, username);
            ResultSet resultSet = statement.executeQuery();
            return resultSet.isBeforeFirst();
        } catch (SQLException e) {
            System.out.println("Error occurred when checking username");
            e.printStackTrace();
        } finally {
            cm.closeConnection();
        }
        return true;
    }

    private static void loginPatient(String[] tokens) {
        // TODO: Part 1
        if (currentPatient != null || currentCaregiver != null) {
            System.out.println("User already logged in!");
            return;
        }
        if (tokens.length != 3) {
            System.out.println("Login failed.");
            return;
        }

        String username = tokens[1];
        String password = tokens[2];

        ConnectionManager cm = new ConnectionManager();
        Connection con = cm.createConnection();

        String getSalt = "SELECT Salt FROM Patients WHERE Username = ?";
        try {
            PreparedStatement statement = con.prepareStatement(getSalt);
            statement.setString(1, username);
            ResultSet resultSet = statement.executeQuery();
            if (!resultSet.next()) {
                System.out.println("Login failed.");
                return;
            }

            byte[] salt = resultSet.getBytes("Salt");
            byte[] hash = Util.generateHash(password, salt);

            String login = "SELECT * FROM Patients WHERE Username = ? and Hash = ?";
            PreparedStatement loginStatement = con.prepareStatement(login);
            loginStatement.setString(1, username);
            loginStatement.setBytes(2, hash);
            ResultSet loginResult = loginStatement.executeQuery();
            if (!loginResult.next()) {
                System.out.println("Login failed.");
                return;
            }

            currentPatient = new Patient.PatientBuilder(username, salt, hash).build();
            System.out.println("Logged in as: " + username);
        } catch (SQLException e) {
            System.out.println("Login failed.");
            e.printStackTrace();
        } finally {
            cm.closeConnection();
        }
    }

    private static void loginCaregiver(String[] tokens) {
        if (currentPatient != null || currentCaregiver != null) {
            System.out.println("User already logged in!");
            return;
        }
        if (tokens.length != 3) {
            System.out.println("Login failed.");
            return;
        }

        String username = tokens[1];
        String password = tokens[2];

        ConnectionManager cm = new ConnectionManager();
        Connection con = cm.createConnection();

        String getSalt = "SELECT Salt FROM Caregivers WHERE Username = ?";
        try {
            PreparedStatement statement = con.prepareStatement(getSalt);
            statement.setString(1, username);
            ResultSet resultSet = statement.executeQuery();
            if (!resultSet.next()) {
                System.out.println("Login failed.");
                return;
            }

            byte[] salt = resultSet.getBytes("Salt");
            byte[] hash = Util.generateHash(password, salt);

            String login = "SELECT * FROM Caregivers WHERE Username = ? and Hash = ?";
            PreparedStatement loginStatement = con.prepareStatement(login);
            loginStatement.setString(1, username);
            loginStatement.setBytes(2, hash);
            ResultSet loginResult = loginStatement.executeQuery();
            if (!loginResult.next()) {
                System.out.println("Login failed.");
                return;
            }

            currentCaregiver = new Caregiver.CaregiverBuilder(username, salt, hash).build();
            System.out.println("Logged in as: " + username);
        } catch (SQLException e) {
            System.out.println("Login failed.");
            e.printStackTrace();
        } finally {
            cm.closeConnection();
        }
    }

    private static void uploadAvailability(String[] tokens) {
        if (currentCaregiver == null) {
            System.out.println("Please login as a caregiver first!");
            return;
        }

        if (tokens.length != 2) {
            System.out.println("Please try again!");
            return;
        }

        String date = tokens[1];
        Date d;
        try {
            d = Date.valueOf(date);
        } catch (IllegalArgumentException e) {
            System.out.println("Please enter a valid date!");
            return;
        }

        try {
            currentCaregiver.uploadAvailability(d);
            System.out.println("Availability uploaded!");
        } catch (SQLException e) {
            System.out.println("Failed to upload availability.");
            e.printStackTrace();
        }
    }

    private static void addDoses(String[] tokens) {
        if (tokens.length != 3) {
            System.out.println("Please try again!");
            return;
        }

        String vaccineName = tokens[1];
        int doses;
        try {
            doses = Integer.parseInt(tokens[2]);
        } catch (NumberFormatException e) {
            System.out.println("Please try again!");
            return;
        }

        try {
            Vaccine vaccine = new Vaccine.VaccineGetter(vaccineName).get();
            vaccine.increaseAvailableDoses(doses);
            System.out.println("Doses updated!");
        } catch (SQLException e) {
            System.out.println("Error occurred when updating number of doses");
            e.printStackTrace();
        }
    }

    private static void showAppointments(String[] tokens) {
        // TODO: Part 2
    }

    private static void logout(String[] tokens) {
        if (currentCaregiver != null) {
            currentCaregiver = null;
        } else if (currentPatient != null) {
            currentPatient = null;
        } else {
            System.out.println("No user is logged in.");
            return;
        }

        System.out.println("Successfully logged out.");
    }
}


Patient logic:

In [ ]:
package scheduler.model;


import java.sql.*;

import java.util.Arrays;

import scheduler.db.ConnectionManager;

import scheduler.util.Util;


public class Patient {

    private final String username;

    private final byte[] salt;

    private final byte[] hash;


    private Patient(PatientBuilder builder) {

        this.username = builder.username;

        this.salt = builder.salt;

        this.hash = builder.hash;

    }


    private Patient(PatientGetter getter) {

        this.username = getter.username;

        this.salt = getter.salt;

        this.hash = getter.hash;

    }


    // Getters

    public String getUsername() {

        return username;

    }


    public byte[] getSalt() {

        return salt;

    }


    public byte[] getHash() {

        return hash;

    }


    public void saveToDB() throws SQLException {

        scheduler.db.ConnectionManager cm = new scheduler.db.ConnectionManager();

        Connection con = cm.createConnection();


        String addPatient = "INSERT INTO Patients VALUES (? , ?, ?)";

        try {

            PreparedStatement statement = con.prepareStatement(addPatient);

            statement.setString(1, this.username);

            statement.setBytes(2, this.salt);

            statement.setBytes(3, this.hash);

            statement.executeUpdate();

        } catch (SQLException e) {

            throw new SQLException();

        } finally {

            cm.closeConnection();

        }

    }


    public void uploadAvailability(Date d) throws SQLException {

        scheduler.db.ConnectionManager cm = new scheduler.db.ConnectionManager();

        Connection con = cm.createConnection();


        String addAvailability = "INSERT INTO Availabilities VALUES (? , ?)";

        try {

            PreparedStatement statement = con.prepareStatement(addAvailability);

            statement.setDate(1, d);

            statement.setString(2, this.username);

            statement.executeUpdate();

        } catch (SQLException e) {

            throw new SQLException();

        } finally {

            cm.closeConnection();

        }

    }


    public static class PatientBuilder {

        private final String username;

        private final byte[] salt;

        private final byte[] hash;


        public PatientBuilder(String username, byte[] salt, byte[] hash) {

            this.username = username;

            this.salt = salt;

            this.hash = hash;

        }


        public Patient build() {

            return new Patient(this);

        }

    }


    public static class PatientGetter {

        private final String username;

        private final String password;

        private byte[] salt;

        private byte[] hash;


        public PatientGetter(String username, String password) {

            this.username = username;

            this.password = password;

        }


        public Patient get() throws SQLException {

            scheduler.db.ConnectionManager cm = new scheduler.db.ConnectionManager();

            Connection con = cm.createConnection();


            String getPatient = "SELECT Salt, Hash FROM Patients WHERE Username = ?";

            try {

                PreparedStatement statement = con.prepareStatement(getPatient);

                statement.setString(1, this.username);

                ResultSet resultSet = statement.executeQuery();

                while (resultSet.next()) {

                    byte[] salt = resultSet.getBytes("Salt");

                    // we need to call Util.trim() to get rid of the paddings,

                    // try to remove the use of Util.trim() and you'll see :)

                    byte[] hash = scheduler.util.Util.trim(resultSet.getBytes("Hash"));

                    // check if the password matches

                    byte[] calculatedHash = scheduler.util.Util.generateHash(password, salt);

                    if (!Arrays.equals(hash, calculatedHash)) {

                        return null;

                    } else {

                        this.salt = salt;

                        this.hash = hash;

                        return new Patient(this);

                    }

                }

                return null;

            } catch (SQLException e) {

                throw new SQLException();

            } finally {

                cm.closeConnection();

            }

        }

    }

}


Vaccine logic:

In [ ]:
package scheduler.model;


import scheduler.db.ConnectionManager;


import java.sql.Connection;

import java.sql.PreparedStatement;

import java.sql.ResultSet;

import java.sql.SQLException;


public class Vaccine {

    private final String vaccineName;

    private int availableDoses;


    private Vaccine(VaccineBuilder builder) {

        this.vaccineName = builder.vaccineName;

        this.availableDoses = builder.availableDoses;

    }


    private Vaccine(VaccineGetter getter) {

        this.vaccineName = getter.vaccineName;

        this.availableDoses = getter.availableDoses;

    }


    // Getters

    public String getVaccineName() {

        return vaccineName;

    }


    public int getAvailableDoses() {

        return availableDoses;

    }


    public void saveToDB() throws SQLException {

        ConnectionManager cm = new ConnectionManager();

        Connection con = cm.createConnection();


        String addDoses = "INSERT INTO vaccines VALUES (?, ?)";

        try {

            PreparedStatement statement = con.prepareStatement(addDoses);

            statement.setString(1, this.vaccineName);

            statement.setInt(2, this.availableDoses);

            statement.executeUpdate();

        } catch (SQLException e) {

            throw new SQLException();

        } finally {

            cm.closeConnection();

        }

    }


    // Increment the available doses

    public void increaseAvailableDoses(int num) throws SQLException {

        if (num <= 0) {

            throw new IllegalArgumentException("Argument cannot be negative!");

        }

        this.availableDoses += num;


        ConnectionManager cm = new ConnectionManager();

        Connection con = cm.createConnection();


        String removeAvailability  = "UPDATE vaccines SET Doses = ? WHERE name = ?;";

        try {

            PreparedStatement statement = con.prepareStatement(removeAvailability);

            statement.setInt(1, this.availableDoses);

            statement.setString(2, this.vaccineName);

            statement.executeUpdate();

        } catch (SQLException e) {

            throw new SQLException();

        } finally {

            cm.closeConnection();

        }

    }


    // Decrement the available doses

    public void decreaseAvailableDoses(int num) throws SQLException {

        if (this.availableDoses - num < 0) {

            throw new IllegalArgumentException("Not enough available doses!");

        }

        this.availableDoses -= num;

        ConnectionManager cm = new ConnectionManager();

        Connection con = cm.createConnection();


        String removeAvailability  = "UPDATE vaccines SET Doses = ? WHERE name = ?;";

        try {

            PreparedStatement statement = con.prepareStatement(removeAvailability);

            statement.setInt(1, this.availableDoses);

            statement.setString(2, this.vaccineName);

            statement.executeUpdate();

        } catch (SQLException e) {

            throw new SQLException();

        } finally {

            cm.closeConnection();

        }

    }


    @Override

    public String toString() {

        return "Vaccine{" +

                "vaccineName='" + vaccineName + '\'' +

                ", availableDoses=" + availableDoses +

                '}';

    }


    public static class VaccineBuilder {

        private final String vaccineName;

        private int availableDoses;


        public VaccineBuilder(String vaccineName, int availableDoses) {

            this.vaccineName = vaccineName;

            this.availableDoses = availableDoses;

        }


        public Vaccine build() throws SQLException {

            return new Vaccine(this);

        }

    }


    public static class VaccineGetter {

        private final String vaccineName;

        private int availableDoses;


        public VaccineGetter(String vaccineName) {

            this.vaccineName = vaccineName;

        }


        public Vaccine get() throws SQLException {

            ConnectionManager cm = new ConnectionManager();

            Connection con = cm.createConnection();


            String getVaccine = "SELECT Name, Doses FROM Vaccines WHERE Name = ?";

            try {

                PreparedStatement statement = con.prepareStatement(getVaccine);

                statement.setString(1, this.vaccineName);

                ResultSet resultSet = statement.executeQuery();

                while (resultSet.next()) {

                    this.availableDoses = resultSet.getInt("Doses");

                    return new Vaccine(this);

                }

                return null;

            } catch (SQLException e) {

                throw new SQLException();

            } finally {

                cm.closeConnection();

            }

        }

    }

}
